# POLI JÚNIOR + TOTAL PASS
### REGRESSOR PARA CHURN RATE

Importando bibliotecas necessárias:

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from random import randint
from sklearn.manifold import TSNE
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from futures3.thread import ThreadPoolExecutor
from tqdm import tqdm
from pymannkendall import original_test
import pytz
from pyarrow.parquet import ParquetFile
import pyarrow as pa
import shap
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
# Evitar warnings:
import warnings
warnings.filterwarnings("ignore")

### Parâmetros do modelo:

In [16]:
DATE_OF_INFERENCE = '2023-11-17' # data de hoje
CUT_OFF_DATE = '2022-11-17' # 1 ano atrás

# Tabelas dinâmicas Total Pass
daily_token_filepath = 'totalpass/Tabelas/daily_token_validation_parquet.parquet'
employees_filepath = 'totalpass/Tabelas/employees.csv'
gyms_filepath = 'totalpass/Tabelas/gyms.csv'
freezes_filepath = 'totalpass/Tabelas/freezes.csv'
subscriptions_filepath = 'totalpass/Tabelas/subscriptions.csv'
subscriptions_features_dataset_filepath = 'totalpass/Tabelas/subscriptions_features_dataset.csv'

# Tabelas fixas Poli Júnior
features_filepath = 'totalpass/Tabelas/features.csv'
tendencia_de_utilizacao_filepath = 'totalpass/Tabelas/tendência_de_utilização.csv'
cluster_filepath = 'totalpass/Tabelas/df_cluster.csv'
dt_interval = [90, 180, 270, 365]

In [4]:
def criar_features(df_token,df_gyms,df_subscriptions):
    df1 = df_token[['validated_at','employee_id','token','gym_id']]
    df2 = df_token[['employee_id','gym_id']]
    df3 = df_token[['employee_id','validated_at_minute']]
    academias_mais_utilizadas = df1.groupby(['employee_id', 'gym_id']).size().reset_index(name='utilizacoes')
    idx = academias_mais_utilizadas.groupby(['employee_id'])['utilizacoes'].transform(max) == academias_mais_utilizadas['utilizacoes']
    academias_mais_utilizadas = academias_mais_utilizadas[idx]

    gyms1 = df_gyms[['gym_id','gym_status']]
    gyms2 = df_gyms[['gym_id','main_modality']]
    gymsdf = pd.merge(gyms2, df2, how= 'right', on= ['gym_id'])
    dfprinc = pd.merge(gyms1, academias_mais_utilizadas, on='gym_id', how='right')

    modalidades_preferidas = gymsdf.groupby(['employee_id', 'main_modality']).size().reset_index(name='num_main_modality')
    idx = modalidades_preferidas.groupby(['employee_id'])['num_main_modality'].transform(max) == modalidades_preferidas['num_main_modality']
    modalidades_preferidas = modalidades_preferidas[idx]
    modalidades_preferidas = modalidades_preferidas.rename(columns={'main_modality': 'pref_modality'})
    gymsprinc = pd.merge(gymsdf, modalidades_preferidas[['employee_id', 'pref_modality', 'num_main_modality']],
              on=['employee_id'], how='left')

    dfprinc = pd.merge(dfprinc, gymsprinc[['employee_id','pref_modality','num_main_modality']], how= 'left', on= ['employee_id'])
    dfprinc = dfprinc.drop_duplicates()
    academias_distintas_por_usuario = df2.groupby('employee_id')['gym_id'].nunique().reset_index(name='num_distinct_gyms')
    distpref = pd.merge(df2, academias_distintas_por_usuario, on='employee_id', how='left')

    dfprinc = pd.merge(dfprinc, distpref[['employee_id','num_distinct_gyms']], how= 'left', on= ['employee_id'])
    dfprinc = dfprinc.drop_duplicates()
    subs1 = df_subscriptions[['employee_id','payment_source']]

    dfprinc = pd.merge(dfprinc, subs1, how= 'left', on= ['employee_id'])
    dfprinc = dfprinc.drop_duplicates()
    df3['validated_at_minute'] = pd.to_datetime(df3['validated_at_minute'])
    hora_mais_frequente_por_usuario = df3.groupby('employee_id')['validated_at_minute'].agg(lambda x: x.dt.hour.mode().iloc[0])
    df3 = pd.merge(df3, hora_mais_frequente_por_usuario, on='employee_id', how='left')
    df3 = df3.rename(columns={'validated_at_minute_y': 'most_frequent_hour'})

    dfprinc = pd.merge(dfprinc, df3[['employee_id','most_frequent_hour']], how= 'left', on= ['employee_id'])
    dfprinc = dfprinc.drop_duplicates()
    df3['Ano'] = df3['validated_at_minute_x'].dt.year
    df3['Semana'] = df3['validated_at_minute_x'].dt.isocalendar().week
    num_utilizacoes_por_semana = df3.groupby(['employee_id', 'Ano', 'Semana']).size().reset_index(name='Num_Utilizacoes')
    media_utilizacoes_por_semana = num_utilizacoes_por_semana.groupby('employee_id')['Num_Utilizacoes'].mean().reset_index(name='uses_per_week')

    dfprinc = pd.merge(dfprinc, media_utilizacoes_por_semana, how= 'left', on= ['employee_id'])

    return dfprinc

In [5]:
def carregando_dataframes():
    daily_token = pd.read_parquet(daily_token_filepath)
    employees = pd.read_csv(employees_filepath)
    gyms = pd.read_csv(gyms_filepath)
    freezes = pd.read_csv(freezes_filepath)
    subscriptions = pd.read_csv(subscriptions_filepath)
    subscriptions_features_dataset = pd.read_csv(subscriptions_features_dataset_filepath)
    df_tendencia_de_utilizacao = pd.read_csv(tendencia_de_utilizacao_filepath)
    df_cluster = pd.read_csv(cluster_filepath)
    features = criar_features(daily_token, gyms, subscriptions)
    return daily_token, employees, gyms, freezes, subscriptions, subscriptions_features_dataset, features, df_tendencia_de_utilizacao, df_cluster

In [6]:
def criar_df_atemporal(subscriptions_features_dataset, features):
    df_final = pd.merge(subscriptions_features_dataset, features, on='employee_id', how='left')

    # Selecionar colunas para clusterização
    col_cluster = ['employee_id','type_mapped', 'gender_maped',
                'Binario_fee', 'Fee', 'NumGymsWithinRadius',
                'NumGymsNearCompany', 'distancia_cliente_empresa',
                'payment_source_y']
    df_atemp = df_final[col_cluster]
    df_atemp['employee_id'] = df_atemp['employee_id'].astype(int)

    return df_atemp

In [7]:
def criar_df_temporal(CUT_OFF_DATE,dt,df_gyms,df_subscriptions,df_token,df_employees,df_freezes):
    def calcular_gym_status(CUT_OFF_DATE,df,gyms):
        today = CUT_OFF_DATE
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        df = df.drop('Unnamed: 0', axis=1)
        df1 = df[df['validated_at'] <= today]
        df2 = df1[['validated_at','employee_id','token','gym_id']]
        academias_mais_utilizadas = df2.groupby(['employee_id', 'gym_id']).size().reset_index(name='utilizacoes')
        idx = academias_mais_utilizadas.groupby(['employee_id'])['utilizacoes'].transform(max) == academias_mais_utilizadas['utilizacoes']
        academias_mais_utilizadas = academias_mais_utilizadas[idx]
        gyms = gyms.drop('Unnamed: 0', axis=1)
        gyms['registered_at'] = pd.to_datetime(gyms['registered_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['activated_at'] = pd.to_datetime(gyms['activated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['last_inactivated_at'] = pd.to_datetime(gyms['last_inactivated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms1 = gyms[gyms['registered_at'] <= today]
        gyms2 = gyms1[['gym_id','gym_status']]
        dfprinc = pd.merge(gyms2, academias_mais_utilizadas, on='gym_id', how='right')
        return dfprinc
    def calcular_age(CUT_OFF_DATE,df2):
        df2['birthdate'] = pd.to_datetime(df2['birthdate'], errors='coerce')
        df2 = df2.dropna(subset=['birthdate'])

        today = pd.to_datetime(CUT_OFF_DATE)

        # iterando o df2 calculando a coluna 'age'
        df2['age'] = df2.apply(lambda row: today.year - row['birthdate'].year - ((today.month < row['birthdate'].month) or ((today.month == row['birthdate'].month) and (today.day < row['birthdate'].day))), axis=1)

        df2 = df2[['employee_id','age']]
        return df2
    def calcular_freezes(CUT_OFF_DATE,df1,df2):
        df1['created_at'] = pd.to_datetime(df1['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")

        df1 = df1[df1['started_at'] < CUT_OFF_DATE]

        distinct_ids_counts = df1.groupby('employee_id')['id'].nunique().reset_index()
        distinct_ids_counts.columns = ['employee_id', 'freezes']

        df2 = pd.merge(df2, distinct_ids_counts, on='employee_id', how='left')
        df2['freezes'].fillna(0, inplace=True)

        df2 = df2[['employee_id', 'freezes']]
    def calcular_upgrade(CUT_OFF_DATE,df2):
        df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        df2['started_at'] = pd.to_datetime(df2['started_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")

        df2 = df2[df2['started_at'] < CUT_OFF_DATE]

        df2['upgrade'] = np.where(df2['migrated_from_subscription_id'].notna(), 1, 0)

        filtered_subscriptions = df2[df2['upgrade'] == 0]

        count_subscriptions_by_employee = filtered_subscriptions.groupby('employee_id')['subscription_id'].count().reset_index()

        count_subscriptions_by_employee = count_subscriptions_by_employee.rename(columns={'subscription_id': 'N° Subscriptions'})

        df2 = pd.merge(df2, count_subscriptions_by_employee, on='employee_id', how='left')

        df2 = df2[['employee_id', 'upgrade','N° Subscriptions']]

        return df2
    def calcular_pref_modality_utilizaçoes(CUT_OFF_DATE,df,gyms):
        today = pd.to_datetime(CUT_OFF_DATE)
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        df = df.drop('Unnamed: 0', axis=1)
        df1 = df[df['validated_at'] <= today]
        gyms = gyms.drop('Unnamed: 0', axis=1)
        gyms['registered_at'] = pd.to_datetime(gyms['registered_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['activated_at'] = pd.to_datetime(gyms['activated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['last_inactivated_at'] = pd.to_datetime(gyms['last_inactivated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms1 = gyms[gyms['registered_at'] <= today]
        df2 = df1[['employee_id','gym_id']]
        gyms2 = gyms1[['gym_id','main_modality']]
        gymsdf = pd.merge(gyms2, df2, how= 'right', on= ['gym_id'])
        modalidades_preferidas = gymsdf.groupby(['employee_id', 'main_modality']).size().reset_index(name='num_main_modality')
        idx1 = modalidades_preferidas.groupby(['employee_id'])['num_main_modality'].transform(max) == modalidades_preferidas['num_main_modality']
        modalidades_preferidas = modalidades_preferidas[idx1]
        modalidades_preferidas = modalidades_preferidas.rename(columns={'main_modality': 'pref_modality'})
        gymsprinc = pd.merge(gymsdf, modalidades_preferidas[['employee_id', 'pref_modality', 'num_main_modality']],
                on=['employee_id'], how='left')
        return gymsprinc
    def calcular_num_distinct_gyms(CUT_OFF_DATE,df):
        today = pd.to_datetime(CUT_OFF_DATE)
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df1 = df[df['validated_at'] <= today]
        df1['validated_at'] = pd.to_datetime(df1['validated_at'])
        df1['created_at'] = pd.to_datetime(df1['created_at'])
        df1 = df1.drop('Unnamed: 0', axis=1)
        df2 = df1[['employee_id','gym_id']]
        academias_distintas_por_usuario = df2.groupby('employee_id')['gym_id'].nunique().reset_index(name='num_distinct_gyms')
        distpref = pd.merge(df2, academias_distintas_por_usuario, on='employee_id', how='left')
        return distpref
    def calcular_uses_per_week(CUT_OFF_DATE,df):
        df['validated_at_minute'] = pd.to_datetime(df['validated_at_minute'],format = "%Y-%m-%dT%H:%M:%S.%fZ")
        df1 = df[['employee_id','validated_at_minute']]
        df1['Ano'] = df1['validated_at_minute'].dt.year
        df1['Semana'] = df1['validated_at_minute'].dt.isocalendar().week
        today = pd.to_datetime(CUT_OFF_DATE)
        df2 = df1[df1['validated_at_minute'] <= today]
        num_utilizacoes_por_semana = df1.groupby(['employee_id', 'Ano', 'Semana']).size().reset_index(name='Num_Utilizacoes')
        media_utilizacoes_por_semana = num_utilizacoes_por_semana.groupby('employee_id')['Num_Utilizacoes'].mean().reset_index(name='uses_per_week')
        df3 = pd.merge(df2, media_utilizacoes_por_semana, how= 'left', on= ['employee_id'])
        return df3
    def calcular_most_frequent_hour(CUT_OFF_DATE,df):
        df['validated_at_minute'] = pd.to_datetime(df['validated_at_minute'], format='%Y-%m-%dT%H:%M:%S.%fZ')
        df1 = df[['employee_id','validated_at_minute']]
        today = pd.to_datetime(CUT_OFF_DATE)
        df2 = df1[df1['validated_at_minute'] <= today]
        hora_mais_frequente_por_usuario = df2.groupby('employee_id')['validated_at_minute'].agg(lambda x: x.dt.hour.mode().iloc[0])
        df3 = pd.merge(df2, hora_mais_frequente_por_usuario, on='employee_id', how='left')
        return df3
    def active_at_date(CUT_OFF_DATE,df2):
        df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        df2['canceled_at'] = pd.to_datetime(df2['canceled_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")

        CUT_OFF_DATE = pd.to_datetime(CUT_OFF_DATE)

        # Selecionando 1 apenas as assinaturas ativas na data de corte
        df2['active_at_date'] = ((df2['created_at'] <= CUT_OFF_DATE) & ((df2['canceled_at'] >= CUT_OFF_DATE) | (df2['canceled_at'].isnull()))).astype(int)
        # Para ser ativo,
        # 1. A assinatura deve ter sido criada antes da data de corte
        # 2. A assinatura deve ter sido cancelada depois da data de corte ou não ter sido cancelada

        df2 = df2[['employee_id','active_at_date']]
        return df2
    def churn_at_dt(CUT_OFF_DATE,dt,df2):
        CUT_OFF_DATE = pd.to_datetime(CUT_OFF_DATE)

        df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        df2['canceled_at'] = pd.to_datetime(df2['canceled_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        print(CUT_OFF_DATE+pd.Timedelta(dt))

        # Selecionando 1 apenas as assinaturas ativas na data de corte
        df2['churn_at_dt'] = ((df2['created_at'] <= CUT_OFF_DATE) & ((df2['canceled_at'] >= CUT_OFF_DATE) & (df2['canceled_at'] <= CUT_OFF_DATE + pd.Timedelta(days = dt)))).astype(int)
        # Para dar churn no gap,
        # 1. A assinatura deve ter sido criada antes da data de corte

        df2 = df2[['employee_id','churn_at_dt']]
        return df2
    def calcular_normalized_stats(CUT_OFF_DATE,df):
        today = CUT_OFF_DATE
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        df = df.drop('Unnamed: 0', axis=1)
        df['year'] = df['validated_at'].dt.year
        df['month'] = df['validated_at'].dt.month
        df1 = df[df['validated_at'] <= today]
        utilizacoes_por_mes = df1.groupby(['employee_id', 'year', 'month']).size().reset_index(name='num_utilizacoes')
        tabela_pivo = utilizacoes_por_mes.pivot(index='employee_id', columns=['year', 'month'], values='num_utilizacoes').fillna(0)
        tabela_pivo['variacao'] = tabela_pivo.diff(axis=1).fillna(0).sum(axis=1)
        resultados_df = pd.DataFrame(columns=['user_id', 'estatistica_teste', 'valor_p', 'h0_rejeitada'])
        def process_row(employee_id, row):
            estatistica_teste, valor_p, h0_rejeitada, tendencia, x, y, z, w, t = original_test(row[:-1])
            return {'user_id': employee_id, 'estatistica_teste': estatistica_teste, 'valor_p': valor_p, 'h0_rejeitada': h0_rejeitada, 'tendencia': tendencia}
        with ThreadPoolExecutor() as executor:
            futures = []
            for employee_id, row in tabela_pivo.iterrows():
                futures.append(executor.submit(process_row, employee_id, row))
            results = [future.result() for future in tqdm(futures, total=len(futures), desc="Processing")]
            resultados_df = pd.DataFrame(results)
        resultados_df1 = resultados_df.drop('valor_p', axis=1)
        resultados_df1.columns = ['employee_id', 'trend', 'p_value', 'normalized_stats']
        return resultados_df1

    # Carregando todas as funções com CUT_OFF_DATE
    df_age = calcular_age(CUT_OFF_DATE,df_employees)
    print('Calculou age')
    df_upgrade = calcular_upgrade(CUT_OFF_DATE,df_subscriptions)
    print('Calculou upgrade')
    df_gym_status = calcular_gym_status(CUT_OFF_DATE,df_token,df_gyms)
    print("Calculou gym status")
    df_pref_modality = calcular_pref_modality_utilizaçoes(CUT_OFF_DATE,df_token,df_gyms)
    print("Calculou pref_modality")
    df_num_distinct_gyms = calcular_num_distinct_gyms(CUT_OFF_DATE,df_token)
    print("Calculou num_distinct_gyms")
    df_uses_per_week = calcular_uses_per_week(CUT_OFF_DATE,df_token)
    print("Calculou uses_per_week")
    df_most_frequent_hour = calcular_most_frequent_hour(CUT_OFF_DATE,df_token)
    print("Calculou most_frequent_hour")
    df_normalized_stats = calcular_normalized_stats(CUT_OFF_DATE,df_token)
    print("Calculou normalized_stats")
    df_active_at_date = active_at_date(CUT_OFF_DATE,df_subscriptions)
    print('Calculou active_at_date')
    df_churn_at_dt = churn_at_dt(CUT_OFF_DATE, dt,df_subscriptions)
    print('Calculou churn_at_dt')

    # Merge nos 3 DataFrames com chave 'employee_id'
    df_temp = pd.merge(df_age, df_upgrade, on='employee_id', how='inner')
    #print("Após o primeiro merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_active_at_date, on='employee_id', how='inner')
    #print("Após o terceiro merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_churn_at_dt, on='employee_id', how='right')
    #print("Após o quarto merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_gym_status, on='employee_id', how='inner')
    #print("Após o quinto merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_pref_modality, on='employee_id', how='inner')
    #print("Após o sexto merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_num_distinct_gyms, on='employee_id', how='inner')
    #print("Após o sétimo merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_uses_per_week, on='employee_id', how='inner')
    #print("Após o oitavo merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_most_frequent_hour, on='employee_id', how='inner')
    #print("Após o nono merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_normalized_stats, on='employee_id', how='inner')
    #print("Após o décimo merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    return df_temp


In [8]:
def criar_df_inferencia(CUT_OFF_DATE,dt,df_gyms,df_subscriptions,df_token,df_employees,df_freezes):
    def calcular_gym_status(CUT_OFF_DATE,df,gyms):
        today = CUT_OFF_DATE
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        df = df.drop('Unnamed: 0', axis=1)
        df1 = df[df['validated_at'] <= today]
        df2 = df1[['validated_at','employee_id','token','gym_id']]
        academias_mais_utilizadas = df2.groupby(['employee_id', 'gym_id']).size().reset_index(name='utilizacoes')
        idx = academias_mais_utilizadas.groupby(['employee_id'])['utilizacoes'].transform(max) == academias_mais_utilizadas['utilizacoes']
        academias_mais_utilizadas = academias_mais_utilizadas[idx]
        gyms = gyms.drop('Unnamed: 0', axis=1)
        gyms['registered_at'] = pd.to_datetime(gyms['registered_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['activated_at'] = pd.to_datetime(gyms['activated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['last_inactivated_at'] = pd.to_datetime(gyms['last_inactivated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms1 = gyms[gyms['registered_at'] <= today]
        gyms2 = gyms1[['gym_id','gym_status']]
        dfprinc = pd.merge(gyms2, academias_mais_utilizadas, on='gym_id', how='right')
        return dfprinc
    def calcular_age(CUT_OFF_DATE,df2):
        df2['birthdate'] = pd.to_datetime(df2['birthdate'], errors='coerce')
        df2 = df2.dropna(subset=['birthdate'])

        today = pd.to_datetime(CUT_OFF_DATE)

        # iterando o df2 calculando a coluna 'age'
        df2['age'] = df2.apply(lambda row: today.year - row['birthdate'].year - ((today.month < row['birthdate'].month) or ((today.month == row['birthdate'].month) and (today.day < row['birthdate'].day))), axis=1)

        df2 = df2[['employee_id','age']]
        return df2
    def calcular_freezes(CUT_OFF_DATE,df1,df2):
        df1['created_at'] = pd.to_datetime(df1['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")

        df1 = df1[df1['started_at'] < CUT_OFF_DATE]

        distinct_ids_counts = df1.groupby('employee_id')['id'].nunique().reset_index()
        distinct_ids_counts.columns = ['employee_id', 'freezes']

        df2 = pd.merge(df2, distinct_ids_counts, on='employee_id', how='left')
        df2['freezes'].fillna(0, inplace=True)

        df2 = df2[['employee_id', 'freezes']]
    def calcular_upgrade(CUT_OFF_DATE,df2):
        df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        df2['started_at'] = pd.to_datetime(df2['started_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")

        df2 = df2[df2['started_at'] < CUT_OFF_DATE]

        df2['upgrade'] = np.where(df2['migrated_from_subscription_id'].notna(), 1, 0)

        filtered_subscriptions = df2[df2['upgrade'] == 0]

        count_subscriptions_by_employee = filtered_subscriptions.groupby('employee_id')['subscription_id'].count().reset_index()

        count_subscriptions_by_employee = count_subscriptions_by_employee.rename(columns={'subscription_id': 'N° Subscriptions'})

        df2 = pd.merge(df2, count_subscriptions_by_employee, on='employee_id', how='left')

        df2 = df2[['employee_id', 'upgrade','N° Subscriptions']]

        return df2
    def calcular_pref_modality_utilizaçoes(CUT_OFF_DATE,df,gyms):
        today = pd.to_datetime(CUT_OFF_DATE)
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        df = df.drop('Unnamed: 0', axis=1)
        df1 = df[df['validated_at'] <= today]
        gyms = gyms.drop('Unnamed: 0', axis=1)
        gyms['registered_at'] = pd.to_datetime(gyms['registered_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['activated_at'] = pd.to_datetime(gyms['activated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['last_inactivated_at'] = pd.to_datetime(gyms['last_inactivated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms1 = gyms[gyms['registered_at'] <= today]
        df2 = df1[['employee_id','gym_id']]
        gyms2 = gyms1[['gym_id','main_modality']]
        gymsdf = pd.merge(gyms2, df2, how= 'right', on= ['gym_id'])
        modalidades_preferidas = gymsdf.groupby(['employee_id', 'main_modality']).size().reset_index(name='num_main_modality')
        idx1 = modalidades_preferidas.groupby(['employee_id'])['num_main_modality'].transform(max) == modalidades_preferidas['num_main_modality']
        modalidades_preferidas = modalidades_preferidas[idx1]
        modalidades_preferidas = modalidades_preferidas.rename(columns={'main_modality': 'pref_modality'})
        gymsprinc = pd.merge(gymsdf, modalidades_preferidas[['employee_id', 'pref_modality', 'num_main_modality']],
                on=['employee_id'], how='left')
        return gymsprinc
    def calcular_num_distinct_gyms(CUT_OFF_DATE,df):
        today = pd.to_datetime(CUT_OFF_DATE)
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df1 = df[df['validated_at'] <= today]
        df1['validated_at'] = pd.to_datetime(df1['validated_at'])
        df1['created_at'] = pd.to_datetime(df1['created_at'])
        df1 = df1.drop('Unnamed: 0', axis=1)
        df2 = df1[['employee_id','gym_id']]
        academias_distintas_por_usuario = df2.groupby('employee_id')['gym_id'].nunique().reset_index(name='num_distinct_gyms')
        distpref = pd.merge(df2, academias_distintas_por_usuario, on='employee_id', how='left')
        return distpref
    def calcular_uses_per_week(CUT_OFF_DATE,df):
        df['validated_at_minute'] = pd.to_datetime(df['validated_at_minute'],format = "%Y-%m-%dT%H:%M:%S.%fZ")
        df1 = df[['employee_id','validated_at_minute']]
        df1['Ano'] = df1['validated_at_minute'].dt.year
        df1['Semana'] = df1['validated_at_minute'].dt.isocalendar().week
        today = pd.to_datetime(CUT_OFF_DATE)
        df2 = df1[df1['validated_at_minute'] <= today]
        num_utilizacoes_por_semana = df1.groupby(['employee_id', 'Ano', 'Semana']).size().reset_index(name='Num_Utilizacoes')
        media_utilizacoes_por_semana = num_utilizacoes_por_semana.groupby('employee_id')['Num_Utilizacoes'].mean().reset_index(name='uses_per_week')
        df3 = pd.merge(df2, media_utilizacoes_por_semana, how= 'left', on= ['employee_id'])
        return df3
    def calcular_most_frequent_hour(CUT_OFF_DATE,df):
        df['validated_at_minute'] = pd.to_datetime(df['validated_at_minute'], format='%Y-%m-%dT%H:%M:%S.%fZ')
        df1 = df[['employee_id','validated_at_minute']]
        today = pd.to_datetime(CUT_OFF_DATE)
        df2 = df1[df1['validated_at_minute'] <= today]
        hora_mais_frequente_por_usuario = df2.groupby('employee_id')['validated_at_minute'].agg(lambda x: x.dt.hour.mode().iloc[0])
        df3 = pd.merge(df2, hora_mais_frequente_por_usuario, on='employee_id', how='left')
        return df3
    def active_at_date(CUT_OFF_DATE,df2):
        df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        df2['canceled_at'] = pd.to_datetime(df2['canceled_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")

        CUT_OFF_DATE = pd.to_datetime(CUT_OFF_DATE)

        # Selecionando 1 apenas as assinaturas ativas na data de corte
        df2['active_at_date'] = ((df2['created_at'] <= CUT_OFF_DATE) & ((df2['canceled_at'] >= CUT_OFF_DATE) | (df2['canceled_at'].isnull()))).astype(int)
        # Para ser ativo,
        # 1. A assinatura deve ter sido criada antes da data de corte
        # 2. A assinatura deve ter sido cancelada depois da data de corte ou não ter sido cancelada

        df2 = df2[['employee_id','active_at_date']]
        return df2
    def calcular_normalized_stats(CUT_OFF_DATE,df):
        today = CUT_OFF_DATE
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        df = df.drop('Unnamed: 0', axis=1)
        df['year'] = df['validated_at'].dt.year
        df['month'] = df['validated_at'].dt.month
        df1 = df[df['validated_at'] <= today]
        utilizacoes_por_mes = df1.groupby(['employee_id', 'year', 'month']).size().reset_index(name='num_utilizacoes')
        tabela_pivo = utilizacoes_por_mes.pivot(index='employee_id', columns=['year', 'month'], values='num_utilizacoes').fillna(0)
        tabela_pivo['variacao'] = tabela_pivo.diff(axis=1).fillna(0).sum(axis=1)
        resultados_df = pd.DataFrame(columns=['user_id', 'estatistica_teste', 'valor_p', 'h0_rejeitada'])
        def process_row(employee_id, row):
            estatistica_teste, valor_p, h0_rejeitada, tendencia, x, y, z, w, t = original_test(row[:-1])
            return {'user_id': employee_id, 'estatistica_teste': estatistica_teste, 'valor_p': valor_p, 'h0_rejeitada': h0_rejeitada, 'tendencia': tendencia}
        with ThreadPoolExecutor() as executor:
            futures = []
            for employee_id, row in tabela_pivo.iterrows():
                futures.append(executor.submit(process_row, employee_id, row))
            results = [future.result() for future in tqdm(futures, total=len(futures), desc="Processing")]
            resultados_df = pd.DataFrame(results)
        resultados_df1 = resultados_df.drop('valor_p', axis=1)
        resultados_df1.columns = ['employee_id', 'trend', 'p_value', 'normalized_stats']
        return resultados_df1

    # Carregando todas as funções com CUT_OFF_DATE
    df_age = calcular_age(CUT_OFF_DATE,df_employees)
    print('Calculou age')
    df_upgrade = calcular_upgrade(CUT_OFF_DATE,df_subscriptions)
    print('Calculou upgrade')
    df_gym_status = calcular_gym_status(CUT_OFF_DATE,df_token,df_gyms)
    print("Calculou gym status")
    df_pref_modality = calcular_pref_modality_utilizaçoes(CUT_OFF_DATE,df_token,df_gyms)
    print("Calculou pref_modality")
    df_num_distinct_gyms = calcular_num_distinct_gyms(CUT_OFF_DATE,df_token)
    print("Calculou num_distinct_gyms")
    df_uses_per_week = calcular_uses_per_week(CUT_OFF_DATE,df_token)
    print("Calculou uses_per_week")
    df_most_frequent_hour = calcular_most_frequent_hour(CUT_OFF_DATE,df_token)
    print("Calculou most_frequent_hour")
    df_normalized_stats = calcular_normalized_stats(CUT_OFF_DATE,df_token)
    print("Calculou normalized_stats")
    df_active_at_date = active_at_date(CUT_OFF_DATE,df_subscriptions)
    print('Calculou active_at_date')

    # Merge nos 3 DataFrames com chave 'employee_id'
    df_temp = pd.merge(df_age, df_upgrade, on='employee_id', how='inner')
    #print("Após o primeiro merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_active_at_date, on='employee_id', how='inner')
    #print("Após o terceiro merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_gym_status, on='employee_id', how='inner')
    #print("Após o quinto merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_pref_modality, on='employee_id', how='inner')
    #print("Após o sexto merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_num_distinct_gyms, on='employee_id', how='inner')
    #print("Após o sétimo merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_uses_per_week, on='employee_id', how='inner')
    #print("Após o oitavo merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_most_frequent_hour, on='employee_id', how='inner')
    #print("Após o nono merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_normalized_stats, on='employee_id', how='inner')
    #print("Após o décimo merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    #print("Após remover duplicatas:", df_temp.shape)

    return df_temp


In [9]:
def merge_dfs(df_atemp, df_temp):
    df = pd.merge(df_atemp, df_temp, on='employee_id', how='left')
    print("Após o merge:", df.shape)

    # Remove duplicatas:
    df = df.drop_duplicates(subset='employee_id', keep='first')
    print("Após remover duplicatas:", df.shape)

    return df

In [10]:
def preprocessa_df(df):
    # dropar nulos com base na coluna 'active_at_date'
    df = df.dropna(subset=['active_at_date'])

    # Substitua active por 1 e inactive por 0 em gym status:
    df['gym_status'] = df['gym_status'].map({'active': 1, 'inactive': 0})
    # Substitua musculação por 1 e qualquer outra categoria por 0 em pref modality:
    df['pref_modality'] = df['pref_modality'].map(lambda x: 1 if x == 'MUSCULAÇÃO' else 0)
    # Substitua payroll por 0, prepaid por 1 e own_wallet por 2 em payment source:
    df['payment_source_y'] = df['payment_source_y'].map({'payroll_deduction': 0, 'prepaid': 1, 'own_wallet': 2,})

    # Preenchendo valores nulos com 0
    df['Binario_fee'].fillna(0, inplace=True)
    df['Fee'].fillna(0, inplace=True)
    df['upgrade'].fillna(0, inplace=True)
    df['N° Subscriptions'].fillna(0, inplace=True)
    df['payment_source_y'].fillna(0, inplace=True)
    df['pref_modality'].fillna(1, inplace=True) # Preenche com musculação

    # Preenchendo valores nulos com a média
    media_distancia = df['distancia_cliente_empresa'].mean()
    media_idade = df['age'].mean()
    media_genero = df['gender_maped'].mean()

    # Preenche os valores NaN com as médias calculadas
    df['distancia_cliente_empresa'].fillna(media_distancia, inplace=True)
    df['age'].fillna(media_idade, inplace=True)
    df['gender_maped'].fillna(media_genero, inplace=True)
    df.dropna(subset=['type_mapped', 'NumGymsWithinRadius'], inplace=True)


    '''
    df = df[['employee_id','churn_at_dt','active_at_date','NumGymsWithinRadius', 'NumGymsNearCompany',
    'distancia_cliente_empresa', 'payment_source_y', 'age', 'upgrade',
    'N° Subscriptions', 'freezes', 'gym_status', 'utilizacoes',
    'pref_modality', 'num_main_modality',
    'num_distinct_gyms',
    'uses_per_week',
    'normalized_stats','type_mapped', 'gender_maped', 'Binario_fee', 'Fee']]
    '''

    df = df[['employee_id','churn_at_dt','active_at_date','NumGymsWithinRadius', 'NumGymsNearCompany',
    'distancia_cliente_empresa', 'payment_source_y', 'age', 'upgrade',
    'N° Subscriptions','type_mapped', 'gender_maped', 'Binario_fee', 'Fee']]

    df = df[df['active_at_date'] == 1]

    df = df.fillna(0)

    return df

In [11]:
def preprocessa_df_inferencia(df):
    # dropar nulos com base na coluna 'active_at_date'
    df = df.dropna(subset=['active_at_date'])

    # Substitua active por 1 e inactive por 0 em gym status:
    df['gym_status'] = df['gym_status'].map({'active': 1, 'inactive': 0})
    # Substitua musculação por 1 e qualquer outra categoria por 0 em pref modality:
    df['pref_modality'] = df['pref_modality'].map(lambda x: 1 if x == 'MUSCULAÇÃO' else 0)
    # Substitua payroll por 0, prepaid por 1 e own_wallet por 2 em payment source:
    df['payment_source_y'] = df['payment_source_y'].map({'payroll_deduction': 0, 'prepaid': 1, 'own_wallet': 2,})

    # Preenchendo valores nulos com 0
    df['Binario_fee'].fillna(0, inplace=True)
    df['Fee'].fillna(0, inplace=True)
    df['upgrade'].fillna(0, inplace=True)
    df['N° Subscriptions'].fillna(0, inplace=True)
    df['payment_source_y'].fillna(0, inplace=True)
    df['pref_modality'].fillna(1, inplace=True) # Preenche com musculação

    # Preenchendo valores nulos com a média
    media_distancia = df['distancia_cliente_empresa'].mean()
    media_idade = df['age'].mean()
    media_genero = df['gender_maped'].mean()

    # Preenche os valores NaN com as médias calculadas
    df['distancia_cliente_empresa'].fillna(media_distancia, inplace=True)
    df['age'].fillna(media_idade, inplace=True)
    df['gender_maped'].fillna(media_genero, inplace=True)
    df.dropna(subset=['type_mapped', 'NumGymsWithinRadius'], inplace=True)


    '''
    df = df[['employee_id','churn_at_dt','active_at_date','NumGymsWithinRadius', 'NumGymsNearCompany',
    'distancia_cliente_empresa', 'payment_source_y', 'age', 'upgrade',
    'N° Subscriptions', 'freezes', 'gym_status', 'utilizacoes',
    'pref_modality', 'num_main_modality',
    'num_distinct_gyms',
    'uses_per_week',
    'normalized_stats','type_mapped', 'gender_maped', 'Binario_fee', 'Fee']]
    '''

    df = df[['employee_id','active_at_date','NumGymsWithinRadius', 'NumGymsNearCompany',
    'distancia_cliente_empresa', 'payment_source_y', 'age', 'upgrade',
    'N° Subscriptions','type_mapped', 'gender_maped', 'Binario_fee', 'Fee']]

    df = df[df['active_at_date'] == 1]
    df = df.drop('active_at_date', axis=1)

    df = df.fillna(0)

    return df

In [18]:
def treinar_modelo(df, clusterizado ,df_cluster, dt):
    # se o modelo for clusterizado, vamos criar 8 modelos diferentes, 1 para cada cluster:
    if clusterizado:
        modelos = []
        for i in range(8):
            # filtrar df apenas para os employees que estão no cluster i
            df_cluster_especifico = df_cluster[df_cluster['cluster'] == i]
            df_inferencia = df[df['employee_id'].isin(df_cluster_especifico['employee_id'])]
            # Definindo as variáveis preditoras e a variável resposta
            X = df_inferencia.drop(['employee_id', 'churn_at_dt','active_at_date'], axis=1)
            # Normalizando os dados
            scaler_clusterizado = MinMaxScaler()
            X = scaler_clusterizado.fit_transform(X)
            y = df_inferencia['churn_at_dt']

            # Dividindo os dados em treino e teste
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            # Métodos de tratamento de desbalanceamento
            methods = ['None','Oversampling', 'Undersampling', 'SMOTE']

            # Definindo o método de desbalanceamento
            if dt < 100:
              method = 'Undersampling'
            else:
              method = 'None'

            if method == 'Oversampling':
                # Aplicando oversampling
                sampler = RandomOverSampler(random_state=42)
            elif method == 'Undersampling':
                # Aplicando undersampling
                sampler = RandomUnderSampler(random_state=42)
            elif method == 'SMOTE':
                # Aplicando SMOTE
                sampler = SMOTE(random_state=42)

                # Aplicando o método de amostragem
                if method != 'None':
                    X_train, y_train = sampler.fit_resample(X_train, y_train)
                else:
                    X_train, y_train = X_train, y_train

            # Instanciando o modelo de nome
            logreg = RandomForestClassifier()

            # Treinando o modelo
            logreg.fit(X_train, y_train)

            # Fazendo previsões
            y_pred = logreg.predict(X_test)
            Y_probabilities = logreg.predict_proba(X_test)[:, 1]  # Probabilidade prevista para a classe positiva

            results = pd.DataFrame({'churn_at_dt': y_test,
                                    'churn probability': Y_probabilities,'prediction': y_pred,'employee_cluster': i})
            # export results with dt and cluster
            results.to_csv(f'results_{dt}_cluster_{i}.csv', index=False)



            # Avaliando o modelo
            print('Acurácia:', accuracy_score(y_test, y_pred))
            print(confusion_matrix(y_test, y_pred))
            print(classification_report(y_test, y_pred))

            modelos.append(logreg)
        return modelos, scaler_clusterizado
    else:
        # Definindo as variáveis preditoras e a variável resposta
        X = df.drop(['employee_id', 'churn_at_dt','active_at_date'], axis=1)
        # Normalizando os dados
        scaler_geral = MinMaxScaler()
        X = scaler_geral.fit_transform(X)
        y = df['churn_at_dt']

        # Dividindo os dados em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Métodos de tratamento de desbalanceamento
        methods = ['None','Oversampling', 'Undersampling', 'SMOTE']

        # Definindo o método de desbalanceamento
        if dt < 100:
          method = 'Undersampling'
        else:
          method = 'None'

        if method == 'Oversampling':
            # Aplicando oversampling
            sampler = RandomOverSampler(random_state=42)
        elif method == 'Undersampling':
            # Aplicando undersampling
            sampler = RandomUnderSampler(random_state=42)
        elif method == 'SMOTE':
            # Aplicando SMOTE
            sampler = SMOTE(random_state=42)

            # Aplicando o método de amostragem
            if method != 'None':
                X_train, y_train = sampler.fit_resample(X_train, y_train)
            else:
                X_train, y_train = X_train, y_train

        # Instanciando o modelo de nome
        logreg = RandomForestClassifier()

        # Treinando o modelo
        logreg.fit(X_train, y_train)

        # Fazendo previsões
        y_pred = logreg.predict(X_test)
        Y_probabilities = logreg.predict_proba(X_test)[:, 1]  # Probabilidade prevista para a classe positiva

        results = pd.DataFrame({'churn_at_dt': y_test, 'churn probability': Y_probabilities,'prediction': y_pred})
        # export results with dt and cluster
        results.to_csv(f'results_{dt}_padrao.csv', index=False)

        # Avaliando o modelo
        print('Acurácia:', accuracy_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print(classification_report(y_test, y_pred))

        return logreg, scaler_geral


In [13]:
def treinar_clusterizador(df_subscriptions_features_dataset,df_tendencia_de_utilizacao,df_features):
    df = pd.merge(df_subscriptions_features_dataset, df_tendencia_de_utilizacao, on='employee_id', how='outer')
    df = pd.merge(df, df_features, on='employee_id', how='outer')
    df = df.drop_duplicates(subset='employee_id', keep='first')
    col_cluster = ['employee_id','type_mapped', 'Age', 'gender_maped', 'upgrade', 'count_subscriptions',
                'numero_de_ids_distintos', 'Binario_fee', 'Fee', 'NumGymsWithinRadius',
                'NumGymsNearCompany', 'distancia_cliente_empresa','normalized_stats',
                'gym_status','utilizacoes','pref_modality','num_distinct_gyms',
                'payment_source_y','most_frequent_hour','uses_per_week']
    df_filtrado = df[col_cluster]

    # Substitua active por 1 e inactive por 0 em gym status:
    df_filtrado['gym_status'] = df_filtrado['gym_status'].map({'active': 1, 'inactive': 0})
    # Substitua musculação por 1 e qualquer outra categoria por 0 em pref modality:
    df_filtrado['pref_modality'] = df_filtrado['pref_modality'].map(lambda x: 1 if x == 'MUSCULAÇÃO' else 0)
    # Substitua payroll por 0, prepaid por 1 e own_wallet por 2 em payment source:
    df_filtrado['payment_source_y'] = df_filtrado['payment_source_y'].map({'payroll_deduction': 0, 'prepaid': 1, 'own_wallet': 2})

    # Preenchendo valores nulos com 0
    df_filtrado['Binario_fee'].fillna(0, inplace=True)
    df_filtrado['Fee'].fillna(0, inplace=True)
    df_filtrado['pref_modality'].fillna(1, inplace=True) # Preenche com musculação

    # Preenchendo valores nulos com a média
    media_distancia = df_filtrado['distancia_cliente_empresa'].mean()
    media_idade = df_filtrado['Age'].mean()
    media_genero = df_filtrado['gender_maped'].mean()

    # Preenche os valores NaN com as médias calculadas
    df_filtrado['distancia_cliente_empresa'].fillna(media_distancia, inplace=True)
    df_filtrado['Age'].fillna(media_idade, inplace=True)
    df_filtrado['gender_maped'].fillna(media_genero, inplace=True)
    df_filtrado.fillna(0, inplace=True)

    # Normalizando df_filtrado sem a coluna 'employee_id'
    scaler_clusterizador = MinMaxScaler()
    df_scaled = scaler_clusterizador.fit_transform(df_filtrado.drop('employee_id', axis=1))

    # Criando um dataframe com a coluna employee_id de df filtrado e as colunas normalizadas
    df_clusterizacao = pd.concat([pd.DataFrame(df_filtrado['employee_id']), pd.DataFrame(df_scaled)], axis=1)

    # Selecionando colunas específicas do df_clusterização
    df_clusterizacao = df_clusterizacao.iloc[:,[1,2,3,4,9,10,11,12,14,15,17,18,19]] # 13 das 19 colunas

    # Realizando uma amostragem de 160 000 dados
    df_selected = pd.DataFrame(df_scaled).sample(n=16000, random_state=42)

    # Selecionando colunas específicas do df_scaled:
    df_selected = df_selected.iloc[:,[0,1,2,3,8,9,10,11,13,14,16,17,18]] # 13 das 19 colunas

    kmeans_algorithm = KMeans(n_clusters=8, n_init=10, random_state=0)
    kmeans_algorithm.fit(df_selected)

    return kmeans_algorithm, df_clusterizacao


In [17]:
def previsao(dt_interval):
    df_token, df_employees, df_gyms, df_freezes, df_subscriptions, df_subscriptions_features_dataset, df_features, df_tendencia_de_utilizacao, df_cluster = carregando_dataframes()
    print('Dataframes carregados')
    df_atemp = criar_df_atemporal(df_subscriptions_features_dataset,df_features)
    print('DF Atemporal criado')
    dfs = []
    for dt in dt_interval:
        df_temp = criar_df_temporal(CUT_OFF_DATE,dt,df_gyms,df_subscriptions,df_token,df_employees,df_freezes)
        print('DF Temporal criado')
        #df_inferencia_temporal = criar_df_inferencia(DATE_OF_INFERENCE,dt,df_gyms,df_subscriptions,df_token,df_employees,df_freezes)
        #df_inferencia = merge_dfs(df_atemp, df_inferencia_temporal)
        #df_inferencia = preprocessa_df_inferencia(df_inferencia)
        print('DF Inferencia criado')
        #clusterizador, df_clusterizacao = treinar_clusterizador(df_subscriptions_features_dataset,df_tendencia_de_utilizacao,df_features)
        print('Clusterizador criado')
        df_treino = merge_dfs(df_atemp, df_temp)
        print('DF Mergeado')
        df_treino = preprocessa_df(df_treino)
        print('DF Preprocessado')
        modelo_geral, scaler_geral= treinar_modelo(df_treino,0,df_cluster, dt)
        print('Modelo Geral Treinado')
        modelos_clusters, scaler_clusterizado = treinar_modelo(df_treino, 1, df_cluster, dt)
        print('Modelos Clusters Treinados')

        """
        # MODELO GERAL:
        employee_list = []
        previsoes_geral = []
        previsoes_cluster = []
        cluster_employee_list = []
        for index, row in df_inferencia.iterrows():
            # verifica o cluster de employee_id e seleciona o modelo correspondente
            try:
              cluster_employee = clusterizador.predict(df_clusterizacao[df_clusterizacao.index==row['employee_id']].values.reshape(1, -1))[0]
            except:
              cluster_employee = 0
            cluster_employee_list.append(cluster_employee)
            employee_list.append(row['employee_id'])
            X_inferencia = row.drop(['employee_id'])
            X_inferencia = X_inferencia.values.reshape(1, -1)
            # Aplicando scalers
            X_inferencia_geral = scaler_geral.transform(X_inferencia)
            X_inferencia_clusterizado = scaler_clusterizado.transform(X_inferencia)

            previsao_geral = modelo_geral.predict_proba(X_inferencia_geral)
            previsao_clusterizado = modelos_clusters[cluster_employee].predict_proba(X_inferencia_clusterizado)
            previsoes_geral.append(previsao_geral[0][1] * 100)
            previsoes_cluster.append(previsao_clusterizado[0][1] * 100)
        df_previsoes = pd.DataFrame({'employee_id': employee_list, 'cluster_employee':cluster_employee_list, f'churn_at_{dt}_geral': previsoes_geral, f'churn_at_{dt}_cluster':previsoes_cluster})
        dfs.append(df_previsoes)

    # Inicialize o DataFrame final com o primeiro DataFrame na lista dfs
    df_final = dfs[0]

    # Loop sobre os DataFrames restantes na lista dfs
    for df in dfs[1:]:
        # Mesclar o DataFrame atual com o DataFrame final usando 'employee_id' como chave de junção
        df_final = pd.merge(df_final, df, on='employee_id')

        """
    df_final = pd.DataFrame()
    return df_final

Use a célula abaixo para, só com ela, rodar todas as funções uma por uma.

A função previsao é uma função orquestradora, que vai chamando cada função, uma por uma.

In [19]:
# Executando a previsão
df_final = previsao(dt_interval)
display(df_final)

Dataframes carregados
DF Atemporal criado
Calculou age
Calculou upgrade
Calculou gym status
Calculou pref_modality
Calculou num_distinct_gyms
Calculou uses_per_week
Calculou most_frequent_hour


Processing: 100%|██████████| 407769/407769 [07:32<00:00, 902.14it/s]


Calculou normalized_stats
Calculou active_at_date
2022-11-17 00:00:00.000000090
Calculou churn_at_dt
DF Temporal criado
DF Inferencia criado
Clusterizador criado
Após o merge: (1764656, 32)
Após remover duplicatas: (1136775, 32)
DF Mergeado
DF Preprocessado
Acurácia: 0.7672029851943988
[[37199  1645]
 [ 9959  1043]]
              precision    recall  f1-score   support

         0.0       0.79      0.96      0.87     38844
         1.0       0.39      0.09      0.15     11002

    accuracy                           0.77     49846
   macro avg       0.59      0.53      0.51     49846
weighted avg       0.70      0.77      0.71     49846

Modelo Geral Treinado
Acurácia: 0.7981708262284917
[[4933  186]
 [1116  216]]
              precision    recall  f1-score   support

         0.0       0.82      0.96      0.88      5119
         1.0       0.54      0.16      0.25      1332

    accuracy                           0.80      6451
   macro avg       0.68      0.56      0.57      6451
weigh

Processing: 100%|██████████| 407769/407769 [07:32<00:00, 900.88it/s]


Calculou normalized_stats
Calculou active_at_date
2022-11-17 00:00:00.000000180
Calculou churn_at_dt
DF Temporal criado
DF Inferencia criado
Clusterizador criado
Após o merge: (1764656, 32)
Após remover duplicatas: (1136775, 32)
DF Mergeado
DF Preprocessado
Acurácia: 0.6078923083095935
[[18845  8116]
 [11429 11456]]
              precision    recall  f1-score   support

         0.0       0.62      0.70      0.66     26961
         1.0       0.59      0.50      0.54     22885

    accuracy                           0.61     49846
   macro avg       0.60      0.60      0.60     49846
weighted avg       0.61      0.61      0.60     49846

Modelo Geral Treinado
Acurácia: 0.6368004960471245
[[2882  902]
 [1441 1226]]
              precision    recall  f1-score   support

         0.0       0.67      0.76      0.71      3784
         1.0       0.58      0.46      0.51      2667

    accuracy                           0.64      6451
   macro avg       0.62      0.61      0.61      6451
weigh

Processing: 100%|██████████| 407769/407769 [07:33<00:00, 899.18it/s]


Calculou normalized_stats
Calculou active_at_date
2022-11-17 00:00:00.000000270
Calculou churn_at_dt
DF Temporal criado
DF Inferencia criado
Clusterizador criado
Após o merge: (1764656, 32)
Após remover duplicatas: (1136775, 32)
DF Mergeado
DF Preprocessado
Acurácia: 0.6216948200457408
[[ 8890 11177]
 [ 7680 22099]]
              precision    recall  f1-score   support

         0.0       0.54      0.44      0.49     20067
         1.0       0.66      0.74      0.70     29779

    accuracy                           0.62     49846
   macro avg       0.60      0.59      0.59     49846
weighted avg       0.61      0.62      0.61     49846

Modelo Geral Treinado
Acurácia: 0.5968066966361805
[[1364 1477]
 [1124 2486]]
              precision    recall  f1-score   support

         0.0       0.55      0.48      0.51      2841
         1.0       0.63      0.69      0.66      3610

    accuracy                           0.60      6451
   macro avg       0.59      0.58      0.58      6451
weigh

Processing: 100%|██████████| 407769/407769 [07:31<00:00, 903.80it/s]


Calculou normalized_stats
Calculou active_at_date
2022-11-17 00:00:00.000000365
Calculou churn_at_dt
DF Temporal criado
DF Inferencia criado
Clusterizador criado
Após o merge: (1764656, 32)
Após remover duplicatas: (1136775, 32)
DF Mergeado
DF Preprocessado
Acurácia: 0.7020623520442965
[[ 4037 10591]
 [ 4260 30958]]
              precision    recall  f1-score   support

         0.0       0.49      0.28      0.35     14628
         1.0       0.75      0.88      0.81     35218

    accuracy                           0.70     49846
   macro avg       0.62      0.58      0.58     49846
weighted avg       0.67      0.70      0.67     49846

Modelo Geral Treinado
Acurácia: 0.7195783599441947
[[ 400 1410]
 [ 399 4242]]
              precision    recall  f1-score   support

         0.0       0.50      0.22      0.31      1810
         1.0       0.75      0.91      0.82      4641

    accuracy                           0.72      6451
   macro avg       0.63      0.57      0.57      6451
weigh

""
